## API OpenAQ

Este archivo contiene los pasos realizados para la obtención de datos de calidad de aire a través de la API OpenAQ.

In [1]:
import json
import requests
import os
import pymongo

In [14]:
data = data.decode('utf-8')  # Decodificar los bytes a una cadena en formato UTF-8

Conexión con MongoDB para subir los datos extraidos de la API

In [2]:
client = pymongo.MongoClient("mongodb://localhost:27017/")
db = client["ETL_project"]
collection = db["airq"]

Conexión con la API

In [3]:
github = os.getenv("aq_pass")
with open('../aq_pass.txt', 'r') as file:
    token = file.read()
HEADER = {'Authorization': f"password {token}"}

In [20]:
#url = "URL_de_la_API"
#response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print("Error al obtener datos de la API")

### List locations in Spain that measure PM2.5

In [21]:
country_code = "ES" # Españita

parameter = "pm25" # can also use the parameter id, 2  

url = f"https://api.openaq.org/v2/locations?country={country_code}&parameter={parameter}"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.content)

b'{"meta":{"name":"openaq-api","license":"","website":"/","page":1,"limit":100,"found":253},"results":[{"id":1555512,"city":null,"name":"Meteobescano","entity":null,"country":"ES","sources":null,"isMobile":false,"isAnalysis":null,"parameters":[{"id":134,"unit":"%","count":19127,"average":42.5,"lastValue":45.0,"parameter":"humidity","displayName":"humidity %","lastUpdated":"2023-11-04T17:42:27+00:00","parameterId":134,"firstUpdated":"2023-09-10T15:15:58+00:00","manufacturers":null},{"id":125,"unit":"particles/cm\xc2\xb3","count":19127,"average":16.855,"lastValue":3.39,"parameter":"um003","displayName":"um003 particles/cm\xc2\xb3","lastUpdated":"2023-11-04T17:42:27+00:00","parameterId":125,"firstUpdated":"2023-09-10T15:15:58+00:00","manufacturers":null},{"id":132,"unit":"mb","count":19127,"average":1003.75,"lastValue":977.68,"parameter":"pressure","displayName":"pressure mb","lastUpdated":"2023-11-04T17:42:27+00:00","parameterId":132,"firstUpdated":"2023-09-10T15:15:58+00:00","manufactur

In [23]:
data_dict = json.loads(data)  # Convierte la cadena JSON a un diccionario

In [33]:
data = data_dict['results']  # Extraer la lista de documentos

In [34]:
# Inserta los datos en la colección
collection.insert_many(data)

## Locations by country

In [35]:
country_code = "ES"

url = f"https://api.openaq.org/v2/locations?country={country_code}"

headers = {"Accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.content)

b'{"meta":{"name":"openaq-api","license":"","website":"/","page":1,"limit":100,"found":557},"results":[{"id":921030,"city":null,"name":"C. Restoy, 51, Almeria, Spain","entity":null,"country":"ES","sources":null,"isMobile":false,"isAnalysis":null,"parameters":[{"id":1,"unit":"\xc2\xb5g/m\xc2\xb3","count":72356,"average":5.698174298076543,"lastValue":6.96,"parameter":"pm10","displayName":"pm10 \xc2\xb5g/m\xc2\xb3","lastUpdated":"2023-11-04T22:09:00+00:00","parameterId":1,"firstUpdated":"2023-05-07T23:58:00+00:00","manufacturers":null},{"id":19,"unit":"\xc2\xb5g/m\xc2\xb3","count":82078,"average":5.1387265637324555,"lastValue":5.05,"parameter":"pm1","displayName":"pm1 \xc2\xb5g/m\xc2\xb3","lastUpdated":"2023-11-04T02:04:00+00:00","parameterId":19,"firstUpdated":"2023-05-07T23:58:00+00:00","manufacturers":null},{"id":134,"unit":"%","count":66015,"average":60.629999999999995,"lastValue":60.739998,"parameter":"humidity","displayName":"humidity %","lastUpdated":"2023-11-04T22:09:00+00:00","pa

In [44]:
data2 = response.content

In [47]:
data_dict2 = json.loads(data2)

In [49]:
data2 = data_dict2['results']
data2

[{'id': 921030,
  'city': None,
  'name': 'C. Restoy, 51, Almeria, Spain',
  'entity': None,
  'country': 'ES',
  'sources': None,
  'isMobile': False,
  'isAnalysis': None,
  'parameters': [{'id': 1,
    'unit': 'µg/m³',
    'count': 72356,
    'average': 5.698174298076543,
    'lastValue': 6.96,
    'parameter': 'pm10',
    'displayName': 'pm10 µg/m³',
    'lastUpdated': '2023-11-04T22:09:00+00:00',
    'parameterId': 1,
    'firstUpdated': '2023-05-07T23:58:00+00:00',
    'manufacturers': None},
   {'id': 19,
    'unit': 'µg/m³',
    'count': 82078,
    'average': 5.1387265637324555,
    'lastValue': 5.05,
    'parameter': 'pm1',
    'displayName': 'pm1 µg/m³',
    'lastUpdated': '2023-11-04T02:04:00+00:00',
    'parameterId': 19,
    'firstUpdated': '2023-05-07T23:58:00+00:00',
    'manufacturers': None},
   {'id': 134,
    'unit': '%',
    'count': 66015,
    'average': 60.629999999999995,
    'lastValue': 60.739998,
    'parameter': 'humidity',
    'displayName': 'humidity %',
  

In [51]:
collection.insert_many(data2)

In [52]:
import requests

url = "https://api.openaq.org/v2/locations?limit=100&page=1&offset=0&sort=desc&coordinates=41.38879%2C2.15899&radius=1000&order_by=lastUpdated&dump_raw=false"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"detail":"Connection timed out"}


### PM2.5 Madrid 2022

In [55]:
url = "https://api.openaq.org/v2/locations?limit=100&page=1&offset=0&sort=desc&coordinates=40.4165%2C-3.70256&radius=1000&order_by=lastUpdated&dump_raw=false"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

{"meta":{"name":"openaq-api","license":"","website":"/","page":1,"limit":100,"found":1},"results":[{"id":4318,"city":"Madrid","name":"ES1422A","entity":null,"country":"ES","sources":null,"isMobile":false,"isAnalysis":null,"parameters":[{"id":4,"unit":"µg/m³","count":23490,"average":-855.3653232150294,"lastValue":300.0,"parameter":"co","displayName":"co µg/m³","lastUpdated":"2023-09-26T06:00:00+00:00","parameterId":4,"firstUpdated":"2016-11-18T00:00:00+00:00","manufacturers":null},{"id":6,"unit":"µg/m³","count":22659,"average":4.864604877110288,"lastValue":2.0,"parameter":"so2","displayName":"so2 µg/m³","lastUpdated":"2023-09-26T06:00:00+00:00","parameterId":6,"firstUpdated":"2016-11-18T00:00:00+00:00","manufacturers":null},{"id":5,"unit":"µg/m³","count":22916,"average":31.321562663371513,"lastValue":40.0,"parameter":"no2","displayName":"no2 µg/m³","lastUpdated":"2023-09-26T06:00:00+00:00","parameterId":5,"firstUpdated":"2016-11-18T02:00:00+00:00","manufacturers":null},{"id":3,"unit":"µ

In [56]:
data3 = response.content
data_dict3 = json.loads(data3)
data3 = data_dict3['results']

In [57]:
collection.insert_many(data3)

### PM2.5 Barcelona 2022

In [77]:
url = "https://api.openaq.org/v2/averages?temporal=month&parameters_id=2&date_to=2022-12-31T00%3A00%3A00Z&date_from=2022-01-01T00%3A00%3A00Z&locations_id=276005&spatial=location&limit=100&page=1"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

In [59]:
data4 = response.content
data_dict4 = json.loads(data4)
data4 = data_dict4['results']

In [60]:
collection.insert_many(data4)

### PM2.5 Sevilla 2022

In [61]:
url = "https://api.openaq.org/v2/averages?temporal=month&parameters_id=2&date_to=2022-12-31T00%3A00%3A00Z&date_from=2022-01-01T00%3A00%3A00Z&locations_id=4219&spatial=location&limit=100&page=1"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

In [62]:
data5 = response.content
data_dict5 = json.loads(data5)
data5 = data_dict5['results']

In [63]:
collection.insert_many(data5)

### PM2.5 Valencia 2022

In [64]:
url = "https://api.openaq.org/v2/averages?temporal=month&parameters_id=2&date_to=2022-12-31T00%3A00%3A00Z&date_from=2022-01-01T00%3A00%3A00Z&locations_id=3264&spatial=location&limit=100&page=1"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

In [67]:
data6 = response.content
data_dict6 = json.loads(data6)
data6 = data_dict6['results']

In [68]:
collection.insert_many(data6)

### PM2.5 Bilbao 2022

In [69]:
url = "https://api.openaq.org/v2/averages?temporal=month&parameters_id=2&date_to=2022-12-31T00%3A00%3A00Z&date_from=2022-01-01T00%3A00%3A00Z&locations_id=406370&spatial=location&limit=100&page=1"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

In [70]:
data7 = response.content
data_dict7 = json.loads(data7)
data7 = data_dict7['results']

In [72]:
collection.insert_many(data7)

### PM2.5 Málaga 2022

In [74]:
url = "https://api.openaq.org/v2/averages?temporal=month&parameters_id=2&date_to=2022-12-31T00%3A00%3A00Z&date_from=2022-01-01T00%3A00%3A00Z&locations_id=69693&spatial=location&limit=100&page=1"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

In [75]:
data8 = response.content
data_dict8 = json.loads(data8)
data8 = data_dict8['results']

In [76]:
collection.insert_many(data8)